### Evaluation of the Comparison of solution methods for order planning in maintenance with Google OR-Tools

In [2]:
import csv
import os
import numpy as np
from statistics import mean

#### Get data and list of files

In [ ]:
script_dir = os.path.abspath('')
folder_results = os.path.join(script_dir, "results")
folder_results_matrix_relative = os.path.join(folder_results, "results_matrix_relative")
folder_evaluation = os.path.join(script_dir, "evaluation")
folder_evaluation_for_each_instance = os.path.join(folder_evaluation, "for_each_instance")
folder_evaluation_grouped_by_number_of_orders = os.path.join(folder_evaluation, "grouped_by_number_of_orders")
folder_evaluation_grouped_by_structure_of_orders = os.path.join(folder_evaluation, "grouped_by_structure_of_orders")
folder_evaluation_overall = os.path.join(folder_evaluation, "overall")


file_names = ['homberger_200_C1_2_1-results-matrix-relative.csv', 'homberger_200_R1_2_1-results-matrix-relative.csv', 
              'homberger_200_RC1_2_1-results-matrix-relative.csv', 'solomon-100-c101-results-matrix-relative.csv', 
              'solomon-100-r101-results-matrix-relative.csv', 'solomon-100-rc101-results-matrix-relative.csv', 
              'solomon-25-c101-results-matrix-relative.csv', 'solomon-25-r101-results-matrix-relative.csv', 
              'solomon-25-rc101-results-matrix-relative.csv', 'solomon-50-c101-results-matrix-relative.csv', 
              'solomon-50-r101-results-matrix-relative.csv', 'solomon-50-rc101-results-matrix-relative.csv']


#### Evaluation for each instance

In [144]:
# calculate average for each first solution strategie and for each local search strategie
for file_name in file_names:
    # get time_matrix
    with open(os.path.join(folder_results_matrix_relative, file_name), 'r') as read_obj:
        csv_reader = csv.reader(read_obj)
        time_matrix = []
        for line in csv_reader:
            time_matrix.append([float(x) for x in line])
    # get average results for first solution strategies
    first_solution_strategies = [0] * 16
    for i in range(len(time_matrix)):
        first_solution_strategies[i] = mean(time_matrix[i])
    with open(os.path.join(folder_evaluation_for_each_instance, file_name[:-12] + 'first-solution-evaluation.csv'), "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(first_solution_strategies)
    # get average results for local search strategies
    local_search_strategies = [0] * 6
    for i in range(len(time_matrix[0])):
        local_search_strategies[i] = mean([row[i] for row in time_matrix if row[i] != 0])
    with open(os.path.join(folder_evaluation_for_each_instance, file_name[:-12] + 'local-search-evaluation.csv'), "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(local_search_strategies)


#### Evaluation grouped by number of orders

In [145]:
# calculate average of results grouped by number of orders
number_of_orders_list = [25, 50, 100, 200]
for number_of_orders in number_of_orders_list:
    combination_grouped = [[None for i in range(6)] for j in range(16)]
    first_solution_strategies_grouped = [None] * 16
    local_search_strategies_grouped = [None] * 6
    for file_name in [file_name for file_name in file_names if str(number_of_orders) in file_name]:
        with open(os.path.join(folder_results_matrix_relative, file_name), 'r') as read_obj:
            csv_reader = csv.reader(read_obj)
            time_matrix = []
            for line in csv_reader:
                time_matrix.append([float(x) for x in line])
        with open(os.path.join(folder_evaluation_for_each_instance, file_name[:-12] + 'first-solution-evaluation.csv'), 'r') as read_obj:
            csv_reader = csv.reader(read_obj)
            first_solution_strategies = []
            for line in csv_reader:
                first_solution_strategies.append([float(x) for x in line])
        with open(os.path.join(folder_evaluation_for_each_instance, file_name[:-12] + 'local-search-evaluation.csv'), 'r') as read_obj:
            csv_reader = csv.reader(read_obj)
            local_search_strategies = []
            for line in csv_reader:
                local_search_strategies.append([float(x) for x in line])
        if None in combination_grouped[0]:
            combination_grouped = time_matrix
            first_solution_strategies_grouped = first_solution_strategies
            local_search_strategies_grouped = local_search_strategies
        else:
            combination_grouped = np.mean(np.array([combination_grouped, time_matrix]), axis=0)
            first_solution_strategies_grouped = np.mean(np.array([first_solution_strategies_grouped, first_solution_strategies]), axis=0)
            local_search_strategies_grouped = np.mean(np.array([local_search_strategies_grouped, local_search_strategies]), axis=0)
    for row in range(len(combination_grouped)):
        for column in range(len(combination_grouped[row])):
            if combination_grouped[row][column] < 1.0:
                combination_grouped[row][column] = 0.0
    for i in range(len(first_solution_strategies_grouped[0])):
        if first_solution_strategies_grouped[0][i] < 1.0:
            first_solution_strategies_grouped[0][i] = 0.0
    with open(os.path.join(folder_evaluation_grouped_by_number_of_orders, 'combination-average-' + str(number_of_orders)  + '-orders.csv'), "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(combination_grouped)
    with open(os.path.join(folder_evaluation_grouped_by_number_of_orders, 'first-solution-average-' + str(number_of_orders) + '-orders.csv'), "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(first_solution_strategies_grouped)
    with open(os.path.join(folder_evaluation_grouped_by_number_of_orders, 'local-search-average-' + str(number_of_orders)  + '-orders.csv'), "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(local_search_strategies_grouped)


#### Evaluation grouped by structure of orders

In [18]:
# calculate average of results grouped by structure of orders
structure_of_orders_list = ['c', 'r', 'rc']
for structure_of_orders in structure_of_orders_list:
    combination_grouped = [[None for i in range(6)] for j in range(16)]
    first_solution_strategies_grouped = [None] * 16
    local_search_strategies_grouped = [None] * 6

    for file_name in [file_name for file_name in file_names if "-" + str(structure_of_orders) + "1" in file_name.lower() 
                                                            or "_" + str(structure_of_orders) + "1" in file_name.lower() ]:
        with open(os.path.join(folder_results_matrix_relative, file_name), 'r') as read_obj:
            csv_reader = csv.reader(read_obj)
            time_matrix = []
            for line in csv_reader:
                time_matrix.append([float(x) for x in line])
        with open(os.path.join(folder_evaluation_for_each_instance, file_name[:-12] + 'first-solution-evaluation.csv'), 'r') as read_obj:
            csv_reader = csv.reader(read_obj)
            first_solution_strategies = []
            for line in csv_reader:
                first_solution_strategies.append([float(x) for x in line])
        with open(os.path.join(folder_evaluation_for_each_instance, file_name[:-12] + 'local-search-evaluation.csv'), 'r') as read_obj:
            csv_reader = csv.reader(read_obj)
            local_search_strategies = []
            for line in csv_reader:
                local_search_strategies.append([float(x) for x in line])
        if None in combination_grouped[0]:
            combination_grouped = time_matrix
            first_solution_strategies_grouped = first_solution_strategies
            local_search_strategies_grouped = local_search_strategies
        else:
            combination_grouped = np.mean(np.array([combination_grouped, time_matrix]), axis=0)
            first_solution_strategies_grouped = np.mean(np.array([first_solution_strategies_grouped, first_solution_strategies]), axis=0)
            local_search_strategies_grouped = np.mean(np.array([local_search_strategies_grouped, local_search_strategies]), axis=0)
    for row in range(len(combination_grouped)):
        for column in range(len(combination_grouped[row])):
            if combination_grouped[row][column] < 1.0:
                combination_grouped[row][column] = 0.0
    for i in range(len(first_solution_strategies_grouped[0])):
        if first_solution_strategies_grouped[0][i] < 1.0:
            first_solution_strategies_grouped[0][i] = 0.0
    with open(os.path.join(folder_evaluation_grouped_by_structure_of_orders, 'combination-average-' + str(structure_of_orders)  + '-orders.csv'), "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(combination_grouped)
    with open(os.path.join(folder_evaluation_grouped_by_structure_of_orders, 'first-solution-average-' + str(structure_of_orders) + '-orders.csv'), "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(first_solution_strategies_grouped)
    with open(os.path.join(folder_evaluation_grouped_by_structure_of_orders, 'local-search-average-' + str(structure_of_orders)  + '-orders.csv'), "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(local_search_strategies_grouped)

#### Evaluation overall

In [147]:
# calculate average of results overall
combination_grouped = [[None for i in range(6)] for j in range(16)]
first_solution_strategies_grouped = [None] * 16
local_search_strategies_grouped = [None] * 6
for file_name in file_names:
    with open(os.path.join(folder_results_matrix_relative, file_name), 'r') as read_obj:
        csv_reader = csv.reader(read_obj)
        time_matrix = []
        for line in csv_reader:
            time_matrix.append([float(x) for x in line])
    with open(os.path.join(folder_evaluation_for_each_instance, file_name[:-12] + 'first-solution-evaluation.csv'), 'r') as read_obj:
        csv_reader = csv.reader(read_obj)
        first_solution_strategies = []
        for line in csv_reader:
            first_solution_strategies.append([float(x) for x in line])
    with open(os.path.join(folder_evaluation_for_each_instance, file_name[:-12] + 'local-search-evaluation.csv'), 'r') as read_obj:
        csv_reader = csv.reader(read_obj)
        local_search_strategies = []
        for line in csv_reader:
            local_search_strategies.append([float(x) for x in line])
    if None in combination_grouped[0]:
        combination_grouped = time_matrix
        first_solution_strategies_grouped = first_solution_strategies
        local_search_strategies_grouped = local_search_strategies
    else:
        combination_grouped = np.mean(np.array([combination_grouped, time_matrix]), axis=0)
        first_solution_strategies_grouped = np.mean(np.array([first_solution_strategies_grouped, first_solution_strategies]), axis=0)
        local_search_strategies_grouped = np.mean(np.array([local_search_strategies_grouped, local_search_strategies]), axis=0)
for row in range(len(combination_grouped)):
    for column in range(len(combination_grouped[row])):
        if combination_grouped[row][column] < 1.0:
            combination_grouped[row][column] = 0.0
for i in range(len(first_solution_strategies_grouped[0])):
    if first_solution_strategies_grouped[0][i] < 1.0:
        first_solution_strategies_grouped[0][i] = 0.0
with open(os.path.join(folder_evaluation_overall, 'combination-average-overall-orders.csv'), "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(combination_grouped)
with open(os.path.join(folder_evaluation_overall, 'first-solution-average-overall.csv'), "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(first_solution_strategies_grouped)
with open(os.path.join(folder_evaluation_overall, 'local-search-average-overall-orders.csv'), "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(local_search_strategies_grouped)